# Importação das bibliotecas:

In [1]:
import math
import pandas

# Leitura do Dataframe Final gerado na etapa anterior:

In [2]:
dfFinalCivel = pandas.read_csv('dfFinalCivel.csv', low_memory = False)

dfFinalCivel.head()

,numero,ajuizamento,grau,tribunal,classeProcessual,identificadorMovimento,dataHora,codigoNacional,codigoLocal,codigoPaiNacional
0,11819998010001,19990104000000,G1,TJAC,7,NaN,NaN,NaN,NaN,NaN
1,14319848010001,19840528000000,G1,TJAC,7,NaN,NaN,NaN,NaN,NaN
2,16219928010001,19921125000000,G1,TJAC,7,NaN,NaN,NaN,NaN,NaN
3,24219958010001,19951114000000,G1,TJAC,7,NaN,NaN,NaN,NaN,NaN
4,30519878010002,19870824080000,G1,TJAC,7,0.0,2.009091e+13,132.0,NaN,NaN


# Seleção do tribunal que está sendo analisado:

In [3]:
tribunal = dfFinalCivel.loc[0, 'tribunal']

# Listagem dos processos presentes no Dataframe Final:

In [4]:
listaProcessos = dfFinalCivel['numero']

listaProcessos = sorted(set(listaProcessos))

# Listagem dos códigos de movimento presentes no Dataframe Final:

In [5]:
listaCodigos = []

for i in range(dfFinalCivel.shape[0]):
        
    codigo = dfFinalCivel.loc[i, 'codigoNacional']

    if math.isnan(codigo):

        codigo = dfFinalCivel.loc[i, 'codigoPaiNacional']

    if (not math.isnan(codigo)):

        listaCodigos.append(codigo)
               
listaCodigos = sorted(set(listaCodigos))

listaCodigos = [int(n) for n in listaCodigos]

# Geração da Matriz de Movimentos Predecessores:

In [6]:
matrizPredecessor = pandas.DataFrame(0, index = range(len(listaCodigos)), columns = (['insercao'] + listaCodigos))

matrizPredecessor.loc[:, 'insercao'] = listaCodigos

matrizPredecessor.head()

,insercao,1,3,11,14,18,22,26,36,48,...,980,981,982,985,1013,1045,1051,1059,1061,99999999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
listaColunas = ['codigoNacional', 'codigoPaiNacional']

In [8]:
for numero in listaProcessos:
    
    recorte = dfFinalCivel[dfFinalCivel['numero'] == numero][listaColunas].copy()
    
    recorte.reset_index(drop = True, inplace = True)
    
    for movimento in range(0, (recorte.shape[0] - 1)):
                      
        anterior = recorte.loc[movimento, 'codigoNacional']
        
        if math.isnan(anterior):
            
            anterior = recorte.loc[movimento, 'codigoPaiNacional']
        
        insercao = recorte.loc[(movimento + 1), 'codigoNacional']
        
        if math.isnan(insercao):
            
            insercao = recorte.loc[(movimento + 1), 'codigoPaiNacional']
        
        try:
            
            linha = listaCodigos.index(insercao)
        
            coluna = (listaCodigos.index(anterior) + 1)
        
            # Atualiza a contagem do movimento predecessor:
        
            matrizPredecessor.iloc[linha, coluna] = (matrizPredecessor.iloc[linha, coluna] + 1) 
        
        except:
            
            None
            
    del recorte
    
matrizPredecessor.head()

,insercao,1,3,11,14,18,22,26,36,48,...,980,981,982,985,1013,1045,1051,1059,1061,99999999
0,1,13,6,0,22,0,0,15,0,19,...,0,0,0,2,0,0,7,0,1,0
1,3,15,16,0,64,0,0,35,1,11,...,1,0,0,0,0,0,2,0,11,0
2,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,365,779,0,476,0,2,19,1,1036,...,3,0,0,0,1,1,10,15,54,0
4,18,0,0,0,0,2,0,0,3,0,...,0,0,1,0,0,0,0,0,0,0


# Exportação da Matriz de Movimentos Predecessores em formato CSV:

In [9]:
nomeArquivo = 'matrizPredecessor' + tribunal + '.csv'

matrizPredecessor.to_csv(nomeArquivo, index = False)